In [1]:
import numpy as np
from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score, roc_curve
import matplotlib.pyplot as plt
import pandas as pd
import lightgbm as lgb
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, precision_recall_fscore_support

In [2]:
demo = pd.read_csv('demo_xgb.csv')
notes = pd.read_csv('notes_cnn.csv')
ts = pd.read_csv('ts_gru.csv')

In [3]:
# rename the columns
demo = demo.rename(columns={'Probability': 'demo_prob'})
notes = notes.rename(columns={'Probability': 'notes_prob'})
ts = ts.rename(columns={'Probability': 'ts_prob'})

# merge the DataFrames on 'ID'
fusion_data = pd.merge(demo[['ID', 'demo_prob', 'Mortality']], 
                     notes[['ID', 'notes_prob']], 
                     on='ID')
fusion_data = pd.merge(fusion_data, ts[['ID', 'ts_prob']], 
                     on='ID')

In [4]:
fusion_data.head()

,ID,demo_prob,Mortality,notes_prob,ts_prob
0,360.0,0.542043,0.0,0.629179,0.483322
1,94.0,0.496013,0.0,0.035935,0.443623
2,141.0,0.649976,1.0,0.999956,0.954371
3,330.0,0.243745,0.0,0.000002,0.124060
4,356.0,0.531977,1.0,0.999864,0.558700


In [5]:
best_auc = 0
best_weights = (0, 0, 0)  
best_combined_proba = None 

for w1 in np.linspace(0, 1, num=100):  
    for w2 in np.linspace(0, 1 - w1, num=100):  
        w3 = 1 - w1 - w2  
        combined_proba = w1 * fusion_data['demo_prob'] + w2 * fusion_data['notes_prob'] + w3 * fusion_data['ts_prob']
        auc = roc_auc_score(fusion_data['Mortality'], combined_proba)
        
        if auc > best_auc:
            best_auc = auc
            best_weights = (w1, w2, w3)
            best_combined_proba = combined_proba

print(f"Best AUC: {best_auc} with weights {best_weights[0]} for model 1, {best_weights[1]} for model 2, and {best_weights[2]} for model 3")

Best AUC: 0.8057571214392805 with weights 0.6363636363636365 for model 1, 0.1910009182736455 for model 2, and 0.17263544536271805 for model 3


In [6]:
y_pred = (best_combined_proba > 0.5).astype(int)


precision = precision_score(fusion_data['Mortality'], y_pred)
recall = recall_score(fusion_data['Mortality'], y_pred)
f1 = f1_score(fusion_data['Mortality'], y_pred)

print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

Precision: 0.4163306451612903
Recall: 0.7182608695652174
F1 Score: 0.5271218889597957
